# wdists

In [1]:
from ipynb.fs.full.koselleck import *

In [2]:
dfmodels=get_dfmodels()

Scanning directory for models: 3901it [00:20, 187.39it/s]


In [3]:
def vecs2dist(dfvecs,only_valid_words=True,lim=None):
    if only_valid_words:
        dfvecs=dfvecs.loc[
            set(dfvecs.index)&set(get_valid_words())
        ]
    if lim: dfvecs=dfvecs[:lim]
    dfsim=pd.DataFrame(
        fastdist.cosine_pairwise_distance(
            dfvecs.values.astype(float),
            return_matrix=True
        ),
        index=dfvecs.index,
        columns=dfvecs.index
    )
    dfdist=1-dfsim
    return dfdist

In [4]:
# X=vecs(random.choice(get_default_periods()))
# X

In [5]:
# o=vecs2dist(X)
# o

In [6]:
def do_ingest_dists(obj):
    key,dfperiod,progress,max_runs=obj
    df=None
    dfperiod=dfperiod[dfperiod.run_int<=max_runs]
    iterr=zip(dfperiod.corpus, dfperiod.period, dfperiod.run_int)
    if progress:
        iterr=tqdm(iterr,total=len(dfperiod),position=0)
    for corpus,period,run in iterr:
        dfvecs=vecs(corpus=corpus, period=period, run=run)
        dfdist=vecs2dist(dfvecs)
        if df is None:
            df=dfdist
        else:
            df3=pd.concat([df,dfdist])
            df=df3.groupby(df3.index).mean()
    return (key,df)

In [7]:
for i,dfprd in dfmodels.groupby(['corpus','period']): pass
# dfprd

In [10]:

def ingest_dists(dfmodels,num_proc=1,force=False, progress_data=True, max_runs=10):
    objs = [
        (f'/dists/{corpus}/{period}', gdf, progress_data, max_runs)
        for (corpus,period),gdf in dfmodels.groupby(
            ['corpus','period']
        )
    ]
    iterr=pmap_iter(
        do_ingest_dists,
        objs,
        desc='Ingesting distances into db...',
        num_proc=num_proc
    )
    
    with get_db('wdists',mode='a') as db:
        for key,odf in iterr:
            prefix,key2 = db_split_key(key)
            for i,wordcol in enumerate(odf.columns):
                qstr=f'{wordcol}/{key2}'
                db[qstr]=odf[wordcol]
                if i and not i%100: db.commit()
            db.commit()

In [11]:
# #ingest_dists(dfmodels.query('period_start>=1900 & period_len==5'),num_proc=2,progress_data=False)
# ingest_dists(dfmodels.query('period_start<1700 & period_len==20'),num_proc=2,progress_data=False)
# ingest_dists(dfmodels.query('period_start>=1900 & period_len==20'),num_proc=2,progress_data=False)

In [ ]:
# stop

In [13]:
# for x in dfmodels: print(x)

In [14]:

# def ingest_dists(dfmodels,num_proc=1,force=False):
#     key_grp = [
#         (f'/dists/{corpus}/{period}', gdf)
#         for (corpus,period),gdf in dfmodels.groupby(
#             ['corpus','period']
#         )
#     ]
#     with get_db('wdists',mode='a') as db:
#         if not force:
#             key_grp=[
#                 (k,v) for k,v in key_grp
#                 if k not in db
#             ]
#         iterr=tqdm(key_grp,position=0)
#         for key,grp in iterr:
#             prefix,key2 = db_split_key(key)
#             iterr.set_description(key)
#             odf=do_ingest_dists(grp,progress=True)
#             for wordcol in odf:
#                 qstr=f'{wordcol}/{key2}'
#                 db[qstr]=odf[wordcol]
#                 print(qstr)
#             db.commit()
    
        
# #     keys,grps=zip(*key_grp)
# #     iterr=pmap_iter(
# #         do_ingest_dists,
# #         grps,
# #         num_proc=num_proc
# #     )
# #     for key,res in zip(keys,iterr): dbput(key,res)

In [15]:
# !rm ../db/db.kos2.wdists.sqlite

In [16]:
# dfmodels=get_dfmodels(ybins={5})
# dfmodels

In [17]:
# ingest_dists(dfmodels,force=False,num_proc=4)

In [18]:
# stop

# # Migrate older data-->

# # # import
# # force=False
# # with get_db_shelf('wdists',mode='r') as idb, get_db('wdists',mode='a') as odb:
# # #     for i,(k,v) in enumerate(tqdm(idb.items())): 
# #     for i,k in enumerate(tqdm(idb.keys())): 
# #         if not force and k in odb: continue
# #         odb[k]=idb[k]
# #         if i and not i%1000: odb.commit()
# #     odb.commit()
# # stop

In [19]:
def wdist(word_or_words, period_or_periods=None, corpus_or_corpora=DEFAULT_CORPUS,
          ybin=YBIN_WDIST, ymin=YMIN_WDIST, ymax=YMAX_WDIST, **attrs):
    words,corpora,periods = get_words_corpora_periods(
        word_or_words,period_or_periods,corpus_or_corpora,
        ybin=ybin,ymin=ymin,ymax=ymax
    )
    queries = [f'{w}/{c}/{p}' for w in words for c in corpora for p in periods]
    with get_db('wdists',mode='r') as db:
        d=dict(
            (
                qstr.replace('/','_'),
                db[qstr]
            )
            for qstr in queries
            if qstr in db and len(db[qstr])
        )
        odf=pd.DataFrame(dict((k,v) for k,v in d.items() if len(v)))
        odf['_avg_']=odf.mean(axis=1)
        return odf.sort_values('_avg_').drop('_avg_',1)

In [20]:
# wdist('virtue')

In [21]:
wdist('culture',ymin=1680, ymax=1960, ybin=20)

,culture_bpo_1720-1740,culture_bpo_1740-1760,culture_bpo_1760-1780,culture_bpo_1780-1800,culture_bpo_1800-1820,culture_bpo_1820-1840,culture_bpo_1840-1860,culture_bpo_1860-1880,culture_bpo_1880-1900
cultivation,0.471128,0.283059,0.189279,0.113374,0.107502,0.083780,0.115605,0.241122,0.327860
cultivating,0.329926,0.306642,0.214996,0.269945,0.285167,0.270587,0.354790,0.395134,0.429514
growth,0.504667,0.312152,0.338933,0.289719,0.278823,0.260696,0.260826,0.321123,0.379191
agriculture,0.360903,0.303680,0.293694,0.237383,0.265399,0.344214,0.387082,0.458024,0.561446
husbandry,0.386119,0.346068,0.262702,0.281651,0.294455,0.333633,0.410958,0.547130,0.613107
...,...,...,...,...,...,...,...,...,...
swear,1.353591,1.254379,1.244574,1.289071,1.362386,1.357100,1.352879,1.338404,1.273270
hanged,1.325321,1.337336,1.299556,1.266294,1.313633,1.279444,1.361857,1.395844,1.366821
traitor,1.360921,1.153125,1.404884,1.376912,1.327077,1.301374,1.393291,1.349609,1.295773
prisoner,1.425968,1.426288,1.349219,1.355825,1.289342,1.234941,1.363935,1.448416,1.385728
